# Preprocessing TLC data

This notebook cleans the following datasets: 
1. Yellow taxi data from 2023-06 to 2024-05
2. Green taxi data from 2023-06 to 2024-05


In [188]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [189]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)


In [190]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [191]:
# Read 2023-2024 TLC data
df = spark.read.parquet('../data/landing/tlc_data/*.parquet')


In [192]:
# Read 2023-6 to 2024-5 yellow data
path = "../data/landing/tlc_data"
ydf_2023_6 = spark.read.parquet(path + "/Y-2023-06.parquet")
ydf_2023_7 = spark.read.parquet(path + "/Y-2023-07.parquet")
ydf_2023_8 = spark.read.parquet(path + "/Y-2023-08.parquet")
ydf_2023_9 = spark.read.parquet(path + "/Y-2023-09.parquet")
ydf_2023_10 = spark.read.parquet(path + "/Y-2023-10.parquet")
ydf_2023_11 = spark.read.parquet(path + "/Y-2023-11.parquet")
ydf_2023_12 = spark.read.parquet(path + "/Y-2023-12.parquet")
ydf_2024_1 = spark.read.parquet(path + "/Y-2024-01.parquet")
ydf_2024_2 = spark.read.parquet(path + "/Y-2024-02.parquet")
ydf_2024_3 = spark.read.parquet(path + "/Y-2024-03.parquet")
ydf_2024_4 = spark.read.parquet(path + "/Y-2024-04.parquet")
ydf_2024_5 = spark.read.parquet(path + "/Y-2024-05.parquet")

In [193]:
# Read 2023-6 to 2024-5 green data
path = "../data/landing/tlc_data"
gdf_2023_6 = spark.read.parquet(path + "/G-2023-06.parquet")
gdf_2023_7 = spark.read.parquet(path + "/G-2023-07.parquet")
gdf_2023_8 = spark.read.parquet(path + "/G-2023-08.parquet")
gdf_2023_9 = spark.read.parquet(path + "/G-2023-09.parquet")
gdf_2023_10 = spark.read.parquet(path + "/G-2023-10.parquet")
gdf_2023_11 = spark.read.parquet(path + "/G-2023-11.parquet")
gdf_2023_12 = spark.read.parquet(path + "/G-2023-12.parquet")
gdf_2024_1 = spark.read.parquet(path + "/G-2024-01.parquet")
gdf_2024_2 = spark.read.parquet(path + "/G-2024-02.parquet")
gdf_2024_3 = spark.read.parquet(path + "/G-2024-03.parquet")
gdf_2024_4 = spark.read.parquet(path + "/G-2024-04.parquet")
gdf_2024_5 = spark.read.parquet(path + "/G-2024-05.parquet")

## TLC datasets inspection

In [194]:
# Calculate the total row count for yellow taxi data from 2023-6 to 2024-5
yellow_count = (
    ydf_2023_6.count() + 
    ydf_2023_7.count() + 
    ydf_2023_8.count() + 
    ydf_2023_9.count() + 
    ydf_2023_10.count() + 
    ydf_2023_11.count() + 
    ydf_2023_12.count() + 
    ydf_2024_1.count() + 
    ydf_2024_2.count() + 
    ydf_2024_3.count() + 
    ydf_2024_4.count() + 
    ydf_2024_5.count()
)

# Calculate the total row count for green taxi data from 2023-6 to 2024-5
green_count = (
    gdf_2023_6.count() + 
    gdf_2023_7.count() + 
    gdf_2023_8.count() + 
    gdf_2023_9.count() + 
    gdf_2023_10.count() + 
    gdf_2023_11.count() + 
    gdf_2023_12.count() + 
    gdf_2024_1.count() + 
    gdf_2024_2.count() + 
    gdf_2024_3.count() + 
    gdf_2024_4.count() + 
    gdf_2024_5.count()
)

# Display the green count
print(f"The total green count is {green_count}.")

# Display the yellow count
print(f"The total yellow count is {yellow_count}.")

# Calculate the total row count for all taxi data from 2023-6 to 2024-5
total_count = yellow_count + green_count
# Display the total count
print(f"The total count is {total_count}.")

The total green count is 732489.
The total yellow count is 38916740.
The total count is 39649229.


In [195]:
# Get columns of each DataFrame
columns_ydf = set(ydf_2024_5.columns)
columns_gdf = set(gdf_2024_5.columns)

# Find differences in columns
columns_only_in_df1 = columns_ydf - columns_gdf
columns_only_in_df2 = columns_gdf - columns_ydf

print(f"Columns only in yellowDF: {columns_only_in_df1}")
print(f"Columns only in greenDF: {columns_only_in_df2}")


Columns only in yellowDF: {'Airport_fee', 'tpep_dropoff_datetime', 'tpep_pickup_datetime'}
Columns only in greenDF: {'lpep_pickup_datetime', 'ehail_fee', 'trip_type', 'lpep_dropoff_datetime'}


In [196]:
# Report number of features in each DataFrame
print(f"Number of features in yellowDF: {len(ydf_2024_5.columns)}")
print(f"Number of features in greenDF: {len(gdf_2024_5.columns)}")

Number of features in yellowDF: 19
Number of features in greenDF: 20


## Data Cleaning

In [197]:
# combine the datasets 
ydfs = [ydf_2023_6, ydf_2023_7, ydf_2023_8, ydf_2023_9, ydf_2023_10, ydf_2023_11, ydf_2023_12, 
       ydf_2024_1, ydf_2024_2, ydf_2024_3, ydf_2024_4, ydf_2024_5]
gdfs = [gdf_2023_6, gdf_2023_7, gdf_2023_8, gdf_2023_9, gdf_2023_10, gdf_2023_11, gdf_2023_12, 
       gdf_2024_1, gdf_2024_2, gdf_2024_3, gdf_2024_4, gdf_2024_5]


In [198]:
from pyspark.sql import DataFrame
from functools import reduce

# Function to union two DataFrames
def union_dfs(df1, df2):
    return df1.unionByName(df2)

# Combine all yellow taxi data
yellow_combined = reduce(union_dfs, ydfs)

# Combine all green taxi data
green_combined = reduce(union_dfs, gdfs) 

In [199]:
# Show descriptive statistics for taxi data
yellow_combined.describe().show()
green_combined.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+----------------+---------------------+------------------+--------------------+-------------------+
|summary|          VendorID|   passenger_count|    trip_distance|       RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|       tip_amount|    tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee|
+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+----------------+---------------------+------------------+--------------------+-------------------

+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+---------+---------------------+------------------+------------------+-------------------+--------------------+
|summary|          VendorID|store_and_fwd_flag|       RatecodeID|     PULocationID|     DOLocationID|   passenger_count|     trip_distance|      fare_amount|             extra|            mta_tax|        tip_amount|      tolls_amount|ehail_fee|improvement_surcharge|      total_amount|      payment_type|          trip_type|congestion_surcharge|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+---------+---------------------+------------------+------------------+----

### Unify the columns of the two dataframes

In [200]:
# Drop Airport_fee and trip_type columns from the combined data
yellow_combined = yellow_combined.drop("Airport_fee")
green_combined = green_combined.drop("trip_type")

# Set ehail_fee to 0 for yellow taxi data
yellow_combined = yellow_combined.withColumn("ehail_fee", lit(0))

tpep: Taxicab Passenger Enhancement Program for yellow taxi <br> 
lpep: Livery Passenger Enhancement Program for green taxi

In [201]:
# Rename datetime columns to be consistent
from pyspark.sql.functions import col

yellow_combined = yellow_combined.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                               .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

green_combined = green_combined.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                             .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')


### Merge yellow and green taxi data

In [202]:
# Combine yellow and green taxi data
combined = yellow_combined.unionByName(green_combined)

### Anomaly handling
Filter out anomaly with business logic

In [203]:
# print the schema of the combined data
combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- ehail_fee: double (nullable = true)



In [204]:
from pyspark.sql.functions import min, max

columns_to_check = [
    'passenger_count', 
    'trip_distance', 
    'fare_amount', 
    'extra', 
    'mta_tax', 
    'tip_amount', 
    'tolls_amount', 
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'ehail_fee'
]

# Create a dictionary to store min and max for each column
min_max_dict = {col: df.agg(min(col).alias(f"min_{col}"), max(col).alias(f"max_{col}")).collect()[0] for col in columns_to_check}

# Print the results
for col, values in min_max_dict.items():
    print(f"{col}: Min = {values[f'min_{col}']}, Max = {values[f'max_{col}']}")


passenger_count: Min = 0, Max = 9
trip_distance: Min = 0.0, Max = 345729.44
fare_amount: Min = -1087.3, Max = 386983.63
extra: Min = -39.17, Max = 10002.5
mta_tax: Min = -0.5, Max = 52.09
tip_amount: Min = -330.88, Max = 4174.0
tolls_amount: Min = -91.3, Max = 1702.88
improvement_surcharge: Min = -1.0, Max = 1.0
total_amount: Min = -1094.05, Max = 386987.63
congestion_surcharge: Min = -2.75, Max = 2.75
ehail_fee: Min = None, Max = None


In [212]:
from pyspark.sql.functions import col

# Apply all filters in a single chain
combined = combined.filter(
    # Filter out rows with passenger count greater than 6 or less than 1
    (col("passenger_count").between(1, 6)) &
    # Filter out rows with fare amount less than 3
    (col("fare_amount") >= 3) &
    # Filter out rows with trip distance less than 0.5
    (col("trip_distance") >= 0.5) &
    # Filter out rows with tip amount less than 0 
    (col("tip_amount") >= 0) &
    # Filter out rows with tolls amount less than 0
    (col("tolls_amount") >= 0) &
    # Filter out rows with extra amount less than 0
    (col("extra") >= 0) &
    # Filter out mtax_tax less than 0
    (col("mta_tax") >= 0) &
    # Filter out rows with improvement surcharge less than 0
    (col("improvement_surcharge") >= 0) &
    # Filter out rows with total amount less than 3
    (col("total_amount") >= 3) &
    # Filter out rows with congestion surcharge less than 0
    (col("congestion_surcharge") >= 0)
)


In [213]:
combined.describe().show()

+-------+-------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------+-----------------+------------------+--------------------+------------------+------------------+---------------------+------------------+--------------------+---------+
|summary|           VendorID|   passenger_count|    trip_distance|       RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|       payment_type|      fare_amount|             extra|             mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|ehail_fee|
+-------+-------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------+-----------------+------------------+--------------------+------------------+------------------+---------------------+------------------+--------------------+---------+
|  count|    

In [52]:
# Filter the pick up datetime to between 2023-06 to 2024-05
combined = combined.filter(
    (F.col("pickup_datetime") >= "2023-06-01 00:00:00") & 
    (F.col("pickup_datetime") < "2024-06-01 00:00:00")
)
# Filter the drop off datetime to between 2023-06 to 2024-05
combined = combined.filter(
    (F.col("dropoff_datetime") >= "2023-06-01 00:00:00") & 
    (F.col("dropoff_datetime") < "2024-06-01 00:00:00")
)

In [53]:
# check minimum and maximum datetime
combined.select(F.min("pickup_datetime"), F.max("pickup_datetime")).show()

+--------------------+--------------------+
|min(pickup_datetime)|max(pickup_datetime)|
+--------------------+--------------------+
| 2023-06-01 00:00:00| 2024-05-31 23:58:59|
+--------------------+--------------------+



In [56]:
# Convert to kilometers for the trip distance
combined = combined.withColumn("trip_distance_km", F.col("trip_distance") * 1.60934)
# drop the trip_distance column
combined = combined.drop("trip_distance")

In [64]:
combined = combined.select("pickup_datetime", "dropoff_datetime")
combined.show(5)


+-------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|
+-------------------+-------------------+
|2023-06-01 00:08:48|2023-06-01 00:29:41|
|2023-06-01 00:48:24|2023-06-01 01:07:07|
|2023-06-01 00:54:03|2023-06-01 01:17:29|
|2023-06-01 00:18:44|2023-06-01 00:27:18|
|2023-06-01 00:32:36|2023-06-01 00:45:52|
+-------------------+-------------------+
only showing top 5 rows



In [65]:
combined = combined.withColumn("pickup_str", F.col("pickup_datetime").cast("string"))
combined = combined.withColumn("dropoff_str", F.col("dropoff_datetime").cast("string"))
combined.show(5)


+-------------------+-------------------+-------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|         pickup_str|        dropoff_str|
+-------------------+-------------------+-------------------+-------------------+
|2023-06-01 00:08:48|2023-06-01 00:29:41|2023-06-01 00:08:48|2023-06-01 00:29:41|
|2023-06-01 00:48:24|2023-06-01 01:07:07|2023-06-01 00:48:24|2023-06-01 01:07:07|
|2023-06-01 00:54:03|2023-06-01 01:17:29|2023-06-01 00:54:03|2023-06-01 01:17:29|
|2023-06-01 00:18:44|2023-06-01 00:27:18|2023-06-01 00:18:44|2023-06-01 00:27:18|
|2023-06-01 00:32:36|2023-06-01 00:45:52|2023-06-01 00:32:36|2023-06-01 00:45:52|
+-------------------+-------------------+-------------------+-------------------+
only showing top 5 rows



In [63]:
test_df = test_df.withColumn("pickup_ts", F.to_timestamp("pickup_str"))
test_df = test_df.withColumn("dropoff_ts", F.to_timestamp("dropoff_str"))
test_df = test_df.withColumn("pickup_long", F.col("pickup_ts").cast("long"))
test_df = test_df.withColumn("dropoff_long", F.col("dropoff_ts").cast("long"))
test_df.show(5)


+--------+-------------------+-------------------+---------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+---------+------------------+-------------------+-------------------+-------------------+-------------------+-----------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|ehail_fee|  trip_distance_km|         pickup_str|        dropoff_str|          pickup_ts|         dropoff_ts|pickup_long|dropoff_long|
+--------+-------------------+-------------------+---------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-------

## Add new column ##

## Sampling Data

 ## Datatype Conversion

## Handling duplicates

## Standardization 